## TP6

Una empresa de transporte quiere monitorear la actividad de sus conductores durante el manejo Para ello se propone implementar
un detector de somnolencia en tiempo real tomando como datos de entrada frames provenientes de una cámara que apunta al rosto
del conductor

1. Implementar un detector Haar de caras (vista frontal)

2. En esa ROI detectar los ojos (Se puede usar otro detector Haar

3. Mediante alguna de las técnicas vistas en clase detectar si ambos ojos están abiertos o cerrados y mostrar el estado de la detección sobre el frame


In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import cv2

In [12]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

In [25]:
def hist_compare(img_last, img_current):

    bins_last = cv.calcHist([img_last],[0],None,[256],[0,256])
    bins_current = cv.calcHist([img_current],[0],None,[256],[0,256])
    
    return cv.compareHist(bins_last, bins_current, cv.HISTCMP_CORREL)


In [46]:

cap = cv.VideoCapture(1)
selected_roi = False
show_track = True

fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('output.mp4', fourcc, 5, (int(cap.get(3)), int(cap.get(4))))


first_detection_eyes = False
start_detection = False
relative_eyes = []

ojo1_last = 0
ojo2_last = 0


while(1):

    # 480 x 640
    ret ,frame = cap.read()

    if ret:
          
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        if start_detection:
            # Detect faces in the image
            faces = face_cascade.detectMultiScale(gray, 1.1, minNeighbors=5)#, minSize= (100,130),maxSize=(200,400))

            if len(faces) > 0:
                for (x,y,w,h) in faces:
                    # Le dibujamos un rectángulo amarillo
                    cv.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
                    # Definimos las ROIs en la imagen gris y color
                    roi_gray = gray[y:y+h, x:x+w] 
                    roi_color = frame[y:y+h, x:x+w] 

                    if first_detection_eyes == False:
                        # Para cada rostro hallado le buscamos los ojos
                        eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 8)
                        # En los ojos hallados les dibujamos rectángulos
                        if len(eyes) == 2:

                            first_detection_eyes = True

                            for (ex,ey,ew,eh) in eyes:
                                cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                                relative_eyes.append([ex, ey, ew, eh])
                            
                            eye1= relative_eyes[0]
                            eye2= relative_eyes[1]
                
                            ojo1_last = roi_gray[eye1[1]:eye1[1]+eye1[3], eye1[0]:eye1[0]+eye1[2]]
                            ojo2_last = roi_gray[eye2[1]:eye2[1]+eye2[3], eye2[0]:eye2[0]+eye2[2]]

                            print(f"Posicion relativa de los ojos/cara: {relative_eyes}")

                    else:
                        for eye in relative_eyes:
                            cv.rectangle(roi_color,(eye[0],eye[1]),(eye[0]+eye[2],eye[1]+eye[3]),(0,255,0),2)
                        
                        eye1= relative_eyes[0]
                        eye2= relative_eyes[1]
                        ojo1 = roi_gray[eye1[1]:eye1[1]+eye1[3], eye1[0]:eye1[0]+eye1[2]]
                        ojo2 = roi_gray[eye2[1]:eye2[1]+eye2[3], eye2[0]:eye2[0]+eye2[2]]
                        
                        ojo1_corr = hist_compare(ojo1_last, ojo1)
                        ojo2_corr = hist_compare(ojo2_last, ojo2)
                    
                        correlation = (ojo1_corr + ojo2_corr)/2
                        
                        cv.putText(frame, f'corr_hist = {correlation}', (20, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255 ), 2, cv.LINE_AA)

                        if correlation <= 0.9:
                            cv.putText(frame, 'Deteccion ojos cerrados', (20, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)

                    
        if first_detection_eyes:  
            cv.putText(frame, f'Ojos Detectados', (20, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
            
        
        cv.imshow('Face Recognition', frame)
        out.write(frame)
        
        key = cv.waitKey(10) & 0xFF
            
        if key == ord("q"):
            break
        elif key==ord('s') and ~start_detection:
            start_detection = True
            
    else:
        break

cv.destroyAllWindows()
out.release()
cap.release()

Posicion relativa de los ojos/cara: [[72, 94, 60, 60], [193, 94, 65, 65]]
